In [84]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import SGDRegressor
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.impute import SimpleImputer
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures

import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier 

# Assignment 2 - Regression
### Predict the TOTAL COMPENSATION for this year. 

The data file provided is a salary survey for tech workers in Europe. We want to predict the total amount of compensation they bring in each year, based off of the details of their work. 

Some notes that will be important:
<ul>
    <li>The total compensation will need to be constructed, there is a column for salary, "Yearly brutto salary (without bonus and stocks) in EUR", as well as a column for bonus compensation, "Yearly bonus + stocks in EUR". 
    <li>Some categorical variables will need some work, and there isn't generally an exact answer. The main concern is things with categories that have a bunch of values with a very small count. For example, if there is only 1 person in City X, then that value likely needs to be addressed. We don't want it encoded into a new column of one 1 and thousands of 0s. 
    <li>There is an article exploring some of the data here: https://www.asdcode.de/2021/01/it-salary-survey-december-2020.html
    <li>Imputation and a bit of data manipulation will be required. 
    <li>Use any regression method you'd like. Some ones are closely related to what we've done, you may want to look at them, e.g. ExtraTreesRegressor. 
    <li>Initial accurracy, and potentially final accuracy, may not be great. When I made a plain model will little optimization the errors were large and the R2 was low. There is lots of room for optimization. 
    <li>Research challenge - try some work on the target, look into TransformedTargetRegressor and see if that helps. Recall in stats when we had skewed distributions... Maybe it helps, maybe it doesn't. 
    <li>EDA and data prep are up to you - you'll probably need to do a little exploring to figure out what cleanup is needed. When I did it, I did things kind of iteratively when I did it. For example, look at the value counts, figure out how to treat the different categories, clean something up, look at the results, potentially repeat if needed. After you figure out what needs to be done, you may be able to take some of those steps and incorporate them into a pipeline to be cleaner....
    <li><b>CRITICAL - Please make sure your code runs with RUN ALL. It should load the data that you're given, do all the processing, and spit out results. Comment out or remove anything that you've cleaned up and don't need - e.g. if you scaled a value manually, then moved that into a pipeline, don't leave the original scaling code active when the file is run.</b>
</ul>

### Details and Deliverables

You'll need to build code to produce the predictions. In particular, there's a few things that'll be marked:
<ul>
    <li>Please add a markdown cell at the bottom, and put in a few notes addressing the following:
    <ul>
        <li> Accuracy of your models with/without feature selection. Include both train/test for each. Please use R2 and RMSE. 
        <li> Feature Selection - Please identify what you did for feature selection. No need for a long explaination, something along the lines of "I did X, and the result was that 4 features were removed". Try at least 2 things. 
        <li> Hyperparameter Changes / Grid Search Improvements. What did you try, and why. Similar explaination to above, short. 
        <li> Overall this section should be roughly as long as this intro block - just outline what the results were, what you did to improve, and the results after. 
        <li> If you could use titles/bullet points I'd really appreciate it. 
    </ul>
    <li>Grade Breakdown:
    <ul>
        <li> Code is readable, there are comments: 20%
        <li> Explaination as defined above: 60% (20% each point)
        <li> Accuracy: 20% As compared to everyone else. This will be generously graded, I won't be surprised if overall accuracy is low for most people. 
    </ul>
</ul>

In [85]:
#Load Data
pd.set_option("display.max_rows", None)
df = pd.read_csv("data/Euro_Salary.csv")
df.describe(include="all").T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Timestamp,1253,1248,25/11/2020 08:47:37,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Age,1226.0,NaN,NaN,NaN,32.509788,5.663804,20.0,29.0,32.0,35.0,69.0
Gender,1243,3,Male,1049,NaN,NaN,NaN,NaN,NaN,NaN,NaN
City,1253,119,Berlin,681,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Position,1247,148,Software Engineer,387,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Total years of experience,1237,48,10,138,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Years of experience in Germany,1221,53,2,195,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Seniority level,1241,24,Senior,565,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Your main technology / programming language,1126,256,Java,184,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Other technologies/programming languages you use often,1096,562,Javascript / Typescript,44,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [86]:
df.describe(include="all").T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Timestamp,1253,1248,25/11/2020 08:47:37,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Age,1226.0,NaN,NaN,NaN,32.509788,5.663804,20.0,29.0,32.0,35.0,69.0
Gender,1243,3,Male,1049,NaN,NaN,NaN,NaN,NaN,NaN,NaN
City,1253,119,Berlin,681,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Position,1247,148,Software Engineer,387,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Total years of experience,1237,48,10,138,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Years of experience in Germany,1221,53,2,195,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Seniority level,1241,24,Senior,565,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Your main technology / programming language,1126,256,Java,184,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Other technologies/programming languages you use often,1096,562,Javascript / Typescript,44,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [87]:
df = df.drop(columns='Timestamp')
df = df.drop(columns='Other technologies/programming languages you use often') #this is redundant with because of the main tech column

In [88]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1253 entries, 0 to 1252
Data columns (total 16 columns):
 #   Column                                                  Non-Null Count  Dtype  
---  ------                                                  --------------  -----  
 0   Age                                                     1226 non-null   float64
 1   Gender                                                  1243 non-null   object 
 2   City                                                    1253 non-null   object 
 3   Position                                                1247 non-null   object 
 4   Total years of experience                               1237 non-null   object 
 5   Years of experience in Germany                          1221 non-null   object 
 6   Seniority level                                         1241 non-null   object 
 7   Your main technology / programming language             1126 non-null   object 
 8   Yearly brutto salary (without bonus an

<u><h2>Number of vacation days</u></h2>

In [89]:
df['Number of vacation days'].replace('unlimited', np.nan, inplace=True) #DONE
df['Number of vacation days'].replace('unlimited ', np.nan, inplace=True)
df['Number of vacation days'].replace('Unlimited ', np.nan, inplace=True)
df['Number of vacation days'].replace('Unlimited', np.nan, inplace=True)
df['Number of vacation days'].replace('(no idea)', np.nan, inplace=True)
df['Number of vacation days'].replace('24 labour days', 24, inplace=True)
df['Number of vacation days'].replace('~25', 25, inplace=True)
df['Number of vacation days'].replace('23+', 23, inplace=True)
df['Number of vacation days'].replace('30 in contract (but theoretically unlimited)', 30, inplace=True)


In [90]:

df['Number of vacation days'] = pd.to_numeric(df['Number of vacation days'], errors='coerce')
df['Number of vacation days'].unique()

array([ 30. ,  28. ,  24. ,  29. ,  27. ,   nan,  25. ,  31. ,  26. ,
        60. ,  20. ,  22. ,  38. ,  35. ,  32. ,  40. , 365. ,  36. ,
        23. ,  33. ,  21. ,  14. ,   0. ,  15. ,  16. ,   3. ,  45. ,
        12. ,  50. ,  99. ,  37.5,   1. ,   5. ,  37. ,  39. ,  34. ,
        10. ])

<u><h3>Total Years of Experience</h3></u>

In [91]:
df['Total years of experience'].unique()
df['Total years of experience'] = pd.to_numeric(df['Total years of experience'], errors='coerce')


In [92]:
df['Total years of experience'].replace('1,5', 1.5, inplace=True)  
df['Total years of experience'].replace('1 (as QA Engineer) / 11 in total', 11, inplace=True)  
df['Total years of experience'].replace('2,5', 2.5, inplace=True)  
df['Total years of experience'].replace('15, thereof 8 as CTO', 15, inplace=True) 
df['Total years of experience'].replace('6 (not as a data scientist, but as a lab scientist)', 6, inplace=True) 
df['Total years of experience'].replace('less than year', 0.5, inplace=True)
df['Total years of experience'].replace('383', 3, inplace=True)


<u><h3>Company Type</h3></u>

In [93]:
df['Company type'].value_counts()

Product                                                                     760
Startup                                                                     252
Consulting / Agency                                                         142
Bank                                                                          5
E-commerce                                                                    3
Media                                                                         3
University                                                                    2
Utilities                                                                     2
Fintech                                                                       2
Research                                                                      2
Finance                                                                       2
Corporation                                                                   2
e-commerce                              

In [94]:
df['Company type'].replace('Bank', 'Other', inplace=True)
df['Company type'].replace('E-commerce', 'Other', inplace=True)
df['Company type'].replace('Media', 'Other', inplace=True)
df['Company type'].replace('Fintech', 'Other', inplace=True)
df['Company type'].replace('Finance', 'Other', inplace=True)
df['Company type'].replace('Research', 'Other', inplace=True)
df['Company type'].replace('University', 'Other', inplace=True)
df['Company type'].replace('Corporation', 'Other', inplace=True)
df['Company type'].replace('Utilities', 'Other', inplace=True)
df['Company type'].replace('Retail', 'Other', inplace=True)
df['Company type'].replace('FAANG', 'Other', inplace=True)
df['Company type'].replace('SaaS', 'Other', inplace=True)
df['Company type'].replace('Concern', 'Other', inplace=True)
df['Company type'].replace('Education', 'Other', inplace=True)
df['Company type'].replace('corporate incubator', 'Other', inplace=True)
df['Company type'].replace('Automotive', 'Other', inplace=True)
df['Company type'].replace('service', 'Other', inplace=True)
df['Company type'].replace('eCommerce', 'Other', inplace=True)
df['Company type'].replace('Energy', 'Other', inplace=True)
df['Company type'].replace('Corporation', 'Other', inplace=True)
df['Company type'].replace('Construction', 'Other', inplace=True)
df['Company type'].replace('Bloody enterprise', 'Other', inplace=True)
df['Company type'].replace('Telecommunications', 'Other', inplace=True)
df['Company type'].replace('Full-time position in Education, part-time position in at a data startup', 'Other', inplace=True)
df['Company type'].replace('Industry', 'Other', inplace=True)
df['Company type'].replace('freelance', 'Other', inplace=True)
df['Company type'].replace('Ecommerce', 'Other', inplace=True)
df['Company type'].replace('Cloud', 'Other', inplace=True)
df['Company type'].replace('IT-Outsourcing', 'Other', inplace=True)                                                       
df['Company type'].replace('Game Company', 'Other', inplace=True)
df['Company type'].replace('E-Commerce', 'Other', inplace=True)
df['Company type'].replace('Consulting and Product', 'Other', inplace=True)
df['Company type'].replace('Pharma', 'Other', inplace=True)
df['Company type'].replace('Semiconductor', 'Other', inplace=True)
df['Company type'].replace('e-commerce', 'Other', inplace=True)
df['Company type'].replace('Transport & Logistics', 'Other', inplace=True)
df['Company type'].replace('Big commercial', 'Other', inplace=True)
df['Company type'].replace('Institute', 'Other', inplace=True)
df['Company type'].replace('Non-tech retail', 'Other', inplace=True)
df['Company type'].replace('Bank', 'Other', inplace=True)
df['Company type'].replace('IT Department of established business', 'Other', inplace=True)
df['Company type'].replace('Automotive', 'Other', inplace=True)
df['Company type'].replace('Personal Ltd', 'Other', inplace=True)
df['Company type'].replace('Biergarten', 'Other', inplace=True)
df['Company type'].replace('Publisher', 'Other', inplace=True)
df['Company type'].replace('Research institute', 'Other', inplace=True)
df['Company type'].replace('Science Institute', 'Other', inplace=True)
df['Company type'].replace('Systemhaus', 'Other', inplace=True)
df['Company type'].replace('Publishing and Technology', 'Other', inplace=True)
df['Company type'].replace('Old industry', 'Other', inplace=True)
df['Company type'].replace('Handel', 'Other', inplace=True)
df['Company type'].replace('Outsorce', 'Other', inplace=True)
df['Company type'].replace('Multinational', 'Other', inplace=True)
df['Company type'].replace('Financial', 'Other', inplace=True)
df['Company type'].replace('consumer goods', 'Other', inplace=True)
df['Company type'].replace('Enterprise', 'Other', inplace=True)                                                             
df['Company type'].replace('Behörde', 'Other', inplace=True)
df['Company type'].replace('Outsourse', 'Other', inplace=True)
df['Company type'].replace('Market Research', 'Other', inplace=True)
df['Company type'].replace('Insurance', 'Other', inplace=True)

<u><h3>Company size</h3></u>

In [95]:
df['Company size'].value_counts()

1000+       448
101-1000    405
11-50       174
51-100      147
up to 10     61
Name: Company size, dtype: int64

<u><h3>Main language at work </h3></u>

In [96]:
df['Main language at work'].value_counts()

English               1020
German                 186
Russian                 12
Spanish                  3
Italian                  3
Polish                   2
Czech                    2
Русский                  2
English and German       2
50/50                    1
both                     1
Deuglisch                1
French                   1
Russian, English         1
Name: Main language at work, dtype: int64

In [97]:
df['Main language at work'].replace('Italian', 'Other', inplace=True)
df['Main language at work'].replace('Spanish', 'Other', inplace=True)
df['Main language at work'].replace('Русский', 'Other', inplace=True)
df['Main language at work'].replace('Polish', 'Other', inplace=True)
df['Main language at work'].replace('Czech', 'Other', inplace=True)
df['Main language at work'].replace('English and German', 'English', inplace=True)
df['Main language at work'].replace('French', 'Other', inplace=True)
df['Main language at work'].replace('50/50', 'English', inplace=True)
df['Main language at work'].replace('Deuglisch', 'Other', inplace=True)
df['Main language at work'].replace('both', 'English', inplace=True)
df['Main language at work'].replace('Russian, English', 'English', inplace=True)

<u><h3>Сontract duration</u></h3>

In [98]:
df['Сontract duration'].replace('0', 'Unlimited contract', inplace=True)
df['Сontract duration'].value_counts()

Unlimited contract    1160
Temporary contract      64
Name: Сontract duration, dtype: int64

<u><h3>Employment Status</h3></u>

In [99]:
df['Employment status'].value_counts() 

Full-time employee                                                                 1190
Self-employed (freelancer)                                                           28
Part-time employee                                                                    8
Founder                                                                               3
Working Student                                                                       1
Full-time position, part-time position, & self-employed (freelancing, tutoring)       1
Intern                                                                                1
working student                                                                       1
full-time, but 32 hours per week (it was my request, I'm a student)                   1
Company Director                                                                      1
Werkstudent                                                                           1
Name: Employment status, dtype: 

In [100]:
df['Employment status'].replace('Founder', 'Full-time employee')
df['Employment status'].replace('Company Director', 'Full-time employee')

0                                      Full-time employee
1                                      Full-time employee
2                              Self-employed (freelancer)
3                                      Full-time employee
4                                      Full-time employee
5                                      Full-time employee
6                                      Full-time employee
7                                      Full-time employee
8                                      Full-time employee
9                                      Full-time employee
10                                     Full-time employee
11                             Self-employed (freelancer)
12                                     Full-time employee
13                                     Full-time employee
14                                     Full-time employee
15                                     Full-time employee
16                                     Full-time employee
17            

<u><h3>Yearly bonus + stocks in EUR</u></h3>

In [101]:
df['Yearly bonus + stocks in EUR'] = pd.to_numeric(df['Yearly bonus + stocks in EUR'], errors='coerce')
df['Yearly bonus + stocks in EUR'].value_counts()

0.000000e+00    227
5.000000e+03     56
1.000000e+04     45
2.000000e+03     36
6.000000e+03     26
1.000000e+03     23
3.000000e+03     23
2.000000e+04     21
4.000000e+03     18
1.500000e+04     16
7.000000e+03     15
8.000000e+03     13
7.500000e+04     11
1.200000e+04     11
7.000000e+04     11
3.000000e+04      8
1.010000e+02      8
4.000000e+04      7
1.000000e+05      7
1.500000e+03      7
5.000000e+04      6
8.000000e+04      6
2.500000e+04      6
6.000000e+04      6
5.500000e+04      5
2.000000e+05      5
4.500000e+04      5
5.000000e+02      5
6.500000e+04      5
7.500000e+03      5
1.200000e+05      5
1.100000e+04      4
1.800000e+04      4
9.000000e+04      4
3.500000e+04      4
7.700000e+04      4
9.000000e+03      4
9.500000e+04      4
6.200000e+04      3
1.700000e+04      3
1.500000e+05      3
8.500000e+04      3
4.500000e+03      3
7.300000e+04      3
2.500000e+03      2
1.600000e+04      2
1.400000e+04      2
9.900000e+04      2
7.200000e+04      2
6.500000e+03      2


<u><h3>Your main technology / programming language</h3></u>


In [102]:
df['Your main technology / programming language'].value_counts()

Java                                                            184
Python                                                          164
PHP                                                              56
C++                                                              38
JavaScript                                                       34
Javascript                                                       31
C#                                                               30
python                                                           26
Swift                                                            24
Scala                                                            24
Go                                                               23
Kotlin                                                           22
Python                                                           21
Ruby                                                             19
TypeScript                                      

In [103]:
df['Your main technology / programming language']. replace('JavaScript', 'JavaScript', inplace=True)
df['Your main technology / programming language']. replace('Typescript', 'JavaScript', inplace=True)
df['Your main technology / programming language']. replace('ABAP', 'Other', inplace=True)
df['Your main technology / programming language']. replace('Elixir', 'Other', inplace=True)
df['Your main technology / programming language']. replace('QA', 'Other', inplace=True)
df['Your main technology / programming language']. replace('AWS', 'Other', inplace=True)
df['Your main technology / programming language']. replace('Cloud', 'Other', inplace=True)
df['Your main technology / programming language']. replace('Node.js', 'JavaScript', inplace=True)
df['Your main technology / programming language']. replace('Embedded', 'Other', inplace=True)
df['Your main technology / programming language']. replace('.Net', '.NET', inplace=True)
df['Your main technology / programming language']. replace('React', 'Other', inplace=True)
df['Your main technology / programming language']. replace('Ruby on Rails', 'Other', inplace=True)
df['Your main technology / programming language']. replace('Golang', 'Other', inplace=True)
df['Your main technology / programming language']. replace('Frontend', 'Other', inplace=True)
df['Your main technology / programming language']. replace('yaml', 'Other', inplace=True)
df['Your main technology / programming language']. replace('Angular', 'Other', inplace=True)
df['Your main technology / programming language']. replace('js', 'JavaScript', inplace=True)
df['Your main technology / programming language']. replace('Spark', 'Other', inplace=True)
df['Your main technology / programming language']. replace('PHP', 'PHP', inplace=True)
df['Your main technology / programming language']. replace('NodeJS,' 'JavaScript', inplace=True)
df['Your main technology / programming language']. replace('Bash', 'Other', inplace=True)
df['Your main technology / programming language']. replace('SAP', 'Other', inplace=True)
df['Your main technology / programming language']. replace('C, C++', 'C++', inplace=True)
df['Your main technology / programming language']. replace('Python, SQL', 'Python', inplace=True)
df['Your main technology / programming language']. replace('Java/Kotlin', 'Java', inplace=True)
df['Your main technology / programming language']. replace('C#, .net core', 'C#', inplace=True)
df['Your main technology / programming language']. replace('C/C++', 'C', inplace=True)
df['Your main technology / programming language']. replace('Sql', 'Other', inplace=True)
df['Your main technology / programming language']. replace('JavaScript/Typescript', 'JavaScript', inplace=True)
df['Your main technology / programming language']. replace('Java', 'Java', inplace=True)
df['Your main technology / programming language']. replace('Figma', 'Other', inplace=True)
df['Your main technology / programming language']. replace('JavaScript / TypeScript', 'JavaScript', inplace=True)
df['Your main technology / programming language']. replace('kotlin', 'Kotlin', inplace=True)
df['Your main technology / programming language']. replace('Javascript/Typescript', 'JavaScript', inplace=True)
df['Your main technology / programming language']. replace('Scala / Python', 'Python', inplace=True)
df['Your main technology / programming language']. replace('android', 'Other', inplace=True)
df['Your main technology / programming language']. replace('Objective-C', 'Other', inplace=True)
df['Your main technology / programming language']. replace('Angular, Typescript', 'Other', inplace=True)
df['Your main technology / programming language']. replace('Java, JS', 'Java', inplace=True)
df['Your main technology / programming language']. replace('java/scala/go/clouds/devops', 'Java', inplace=True)
df['Your main technology / programming language']. replace('Kotlin, Java', 'Kotlin', inplace=True)
df['Your main technology / programming language']. replace('Angular, React', 'Other', inplace=True)
df['Your main technology / programming language']. replace('Aws Hadoop Postgre Typescript', 'Other', inplace=True)
df['Your main technology / programming language']. replace('python, scala', 'Python', inplace=True)                                    
df['Your main technology / programming language']. replace('React / JavaScript', 'JavaScript', inplace=True)
df['Your main technology / programming language']. replace('Swift, Objective-C', 'Swft', inplace=True)
df['Your main technology / programming language']. replace('Java/C++', 'Java', inplace=True)
df['Your main technology / programming language']. replace('Kuberenetes, Openstack', 'Kubernetes', inplace=True)
df['Your main technology / programming language']. replace('Network', 'Other', inplace=True)
df['Your main technology / programming language']. replace('Terraform', 'Other', inplace=True)
df['Your main technology / programming language']. replace('Atlassian JIRA', 'Other', inplace=True)
df['Your main technology / programming language']. replace('Ml/Python', 'Python', inplace=True)
df['Your main technology / programming language']. replace('JavaScript, TypeScript', 'JavaScript', inplace=True)
df['Your main technology / programming language']. replace('Power BI', 'Other', inplace=True)
df['Your main technology / programming language']. replace('.net, c++, spss, embeddded', '.NET', inplace=True)
df['Your main technology / programming language']. replace('typescript', 'JavaScript', inplace=True)
df['Your main technology / programming language']. replace('PowerShell', 'Other', inplace=True)
df['Your main technology / programming language']. replace('Kubrrnetes', 'Kubernetes', inplace=True)
df['Your main technology / programming language']. replace('pythin', 'Python', inplace=True)
df['Your main technology / programming language']. replace('Qml', 'Other', inplace=True)
df['Your main technology / programming language']. replace('SAP / ABAP', 'Other', inplace=True)
df['Your main technology / programming language']. replace('Hardware', 'Other', inplace=True)
df['Your main technology / programming language']. replace('Node', 'JavaScript', inplace=True)
df['Your main technology / programming language']. replace('Java/Groovy', 'Java', inplace=True)
df['Your main technology / programming language']. replace('python', 'Python', inplace=True)
df['Your main technology / programming language']. replace('Golang', 'Other', inplace=True)
df['Your main technology / programming language']. replace('Qlik', 'Other', inplace=True)
df['Your main technology / programming language']. replace('Erlang', 'Other', inplace=True)
df['Your main technology / programming language']. replace('Scala, Apache Spark', 'Other', inplace=True)
df['Your main technology / programming language']. replace('Swift, objective-c', 'Swift', inplace=True)
df['Your main technology / programming language']. replace('Test Management', 'Other', inplace=True)
df['Your main technology / programming language']. replace('Spark, Pytorch', 'Other', inplace=True)
df['Your main technology / programming language']. replace('SAS,SQL,Python', 'Other', inplace=True)
df['Your main technology / programming language']. replace('Java, JavaScript', 'Java', inplace=True)
df['Your main technology / programming language']. replace('golang', 'Other', inplace=True)
df['Your main technology / programming language']. replace('Google Cloud Platform', 'Other', inplace=True)
df['Your main technology / programming language']. replace('julia', 'Other', inplace=True)
df['Your main technology / programming language']. replace('Java, terraform', 'Java', inplace=True)
df['Your main technology / programming language']. replace('Nodejs', 'JavaScript', inplace=True)
df['Your main technology / programming language']. replace('AI', 'Other', inplace=True)
df['Your main technology / programming language']. replace('Python, Whole Apache Data Science Stack, AWS', 'Python', inplace=True)
df['Your main technology / programming language']. replace('VB, RPA, Python', 'Other', inplace=True)
df['Your main technology / programming language']. replace('Офмф', 'Other', inplace=True)
df['Your main technology / programming language']. replace('Sql', 'Other', inplace=True)
df['Your main technology / programming language']. replace('JS, Java', 'Java', inplace=True)
df['Your main technology / programming language']. replace('C, Matlab', 'C', inplace=True)
df['Your main technology / programming language']. replace('Business Development Manager Operation', 'Other', inplace=True)
df['Your main technology / programming language']. replace('Azure, SAP', 'Other', inplace=True)
df['Your main technology / programming language']. replace('Cobol', 'Other', inplace=True)
df['Your main technology / programming language']. replace('PM tools', 'Other', inplace=True)
df['Your main technology / programming language']. replace('AWS, GCP, Python,K8s', 'Other', inplace=True)
df['Your main technology / programming language']. replace('Javascript', 'JavaScript', inplace=True)
df['Your main technology / programming language']. replace('React JS', 'JavaScript', inplace=True)
df['Your main technology / programming language']. replace('ML', 'Other', inplace=True)
df['Your main technology / programming language']. replace('TS', 'JavaScript', inplace=True)
df['Your main technology / programming language']. replace('T-SQL', 'Other', inplace=True)
df['Your main technology / programming language']. replace('spark', 'Other', inplace=True)
df['Your main technology / programming language']. replace('c/c++', 'C', inplace=True)
df['Your main technology / programming language']. replace('Angular, TypeScript', 'Other', inplace=True)
df['Your main technology / programming language']. replace('Python, cloud computing', 'Python', inplace=True)
df['Your main technology / programming language']. replace('k8s', 'Other', inplace=True)
df['Your main technology / programming language']. replace('Python (Django)', 'Python', inplace=True)
df['Your main technology / programming language']. replace('TypeScript, JavaScript', 'JavaScript', inplace=True)
df['Your main technology / programming language']. replace('PHP/MySQL', 'PHP', inplace=True)
df['Your main technology / programming language']. replace('swift', 'Swift', inplace=True)
df['Your main technology / programming language']. replace('JavaScript/TypeScript', 'JavaScript', inplace=True)
df['Your main technology / programming language']. replace('Computer Networking,  Network Security', 'Other', inplace=True)
df['Your main technology / programming language']. replace('Terraform, Kubernetes, AWS, GCP, Ansible, Puppet', 'Other', inplace=True)
df['Your main technology / programming language']. replace('Apache Spark', 'Other', inplace=True)
df['Your main technology / programming language']. replace('Terraform', 'Other', inplace=True)
df['Your main technology / programming language']. replace('NodsJs', 'JavaScript', inplace=True)
df['Your main technology / programming language']. replace('Sketch, Figma', 'Other', inplace=True)
df['Your main technology / programming language']. replace('Go/Python', 'Go', inplace=True)
df['Your main technology / programming language']. replace('JavScript', 'JavaScript', inplace=True)
df['Your main technology / programming language']. replace('embedded', 'Other', inplace=True)
df['Your main technology / programming language']. replace('React', 'Other', inplace=True)
df['Your main technology / programming language']. replace('PL/SQL', 'Other', inplace=True)
df['Your main technology / programming language']. replace('c#', 'C#', inplace=True)
df['Your main technology / programming language']. replace('C# .NET', 'C#', inplace=True)
df['Your main technology / programming language']. replace('Frontend: react, node.js', 'Other', inplace=True)
df['Your main technology / programming language']. replace('several', 'Other', inplace=True)
df['Your main technology / programming language']. replace('networking, linux, automation, cloud', 'Other', inplace=True)
df['Your main technology / programming language']. replace('Web', 'Other', inplace=True)
df['Your main technology / programming language']. replace('Linux/UNIX, GIT, Virtualisation Platforms, *shell(s) scripts', 'Other', inplace=True)
df['Your main technology / programming language']. replace('Python/SQL', 'Python', inplace=True)
df['Your main technology / programming language']. replace('ruby on rails', 'Ruby', inplace=True)
df['Your main technology / programming language']. replace('JS, WDIO', 'JavaScript', inplace=True)
df['Your main technology / programming language']. replace('Typescript, Web apps', 'JavaScript', inplace=True)
df['Your main technology / programming language']. replace('Java, .Net', 'Java', inplace=True)
df['Your main technology / programming language']. replace('Kotlin', 'Kotlin', inplace=True)
df['Your main technology / programming language']. replace('C++/C#', 'C++', inplace=True)
df['Your main technology / programming language']. replace('Salesforce', 'Other', inplace=True)
df['Your main technology / programming language']. replace('Typescript', 'JavaScript', inplace=True)
df['Your main technology / programming language']. replace('C++, C#', 'C++', inplace=True)
df['Your main technology / programming language']. replace('Python + SQL', 'Python', inplace=True)
df['Your main technology / programming language']. replace('.Net, Angular', '.NET', inplace=True)
df['Your main technology / programming language']. replace('Perl', 'Other', inplace=True)
df['Your main technology / programming language']. replace('Typescript / Angular', 'JavaScript', inplace=True)
df['Your main technology / programming language']. replace('Apotheker', 'Other', inplace=True)
df['Your main technology / programming language']. replace('Java & PHP', 'Java', inplace=True)
df['Your main technology / programming language']. replace('php, js, python, docker', 'PHP', inplace=True)
df['Your main technology / programming language']. replace('Management', 'Other', inplace=True)
df['Your main technology / programming language']. replace('Python/NLP', 'Python', inplace=True)
df['Your main technology / programming language']. replace('TypeScript/Angular', 'JavaScript', inplace=True)
df['Your main technology / programming language']. replace('Python, database technologies', 'Python', inplace=True)
df['Your main technology / programming language']. replace('Python / JavaScript (React)', 'Python', inplace=True)
df['Your main technology / programming language']. replace('Linux Kernel', 'Other', inplace=True)
df['Your main technology / programming language']. replace('SAP ABAP', 'Other', inplace=True)
df['Your main technology / programming language']. replace('BI, DWH, ETL/ELT', 'Other', inplace=True)
df['Your main technology / programming language']. replace('Haskell', 'Other', inplace=True)
df['Your main technology / programming language']. replace('Blockchain', 'Other', inplace=True)
df['Your main technology / programming language']. replace('Scala, React.js', 'Other', inplace=True)
df['Your main technology / programming language']. replace('sql', 'Other', inplace=True)
df['Your main technology / programming language']. replace('Js, reactJS', 'JavaScript', inplace=True)
df['Your main technology / programming language']. replace('PS, Sketch, React, CSS3', 'Other', inplace=True)
df['Your main technology / programming language']. replace('Linux', 'Other', inplace=True)
df['Your main technology / programming language']. replace('Magento', 'Other', inplace=True)
df['Your main technology / programming language']. replace('React.js / TypeScript', 'Other', inplace=True)
df['Your main technology / programming language']. replace('Go, PHP, SQL', 'Go', inplace=True)
df['Your main technology / programming language']. replace('jenkins bash', 'Other', inplace=True)
df['Your main technology / programming language']. replace('Grails, Groovy', 'Other', inplace=True)
df['Your main technology / programming language']. replace('Pegasystems platform', 'Other', inplace=True)
df['Your main technology / programming language']. replace('SWIFT', 'Swift', inplace=True)
df['Your main technology / programming language']. replace('TypeScript, React', 'JavaScript', inplace=True)
df['Your main technology / programming language']. replace('Sql, BQ, tableau, gtm, ga', 'Other', inplace=True)
df['Your main technology / programming language']. replace('UML', 'Other', inplace=True)
df['Your main technology / programming language']. replace('DC Management', 'Other', inplace=True)
df['Your main technology / programming language']. replace('VHDL', 'Other', inplace=True)
df['Your main technology / programming language']. replace('NodeJS, Typescript, AWS', 'JavaScript', inplace=True)
df['Your main technology / programming language']. replace('Django, Flask, Plotly Dash', 'Other', inplace=True)
df['Your main technology / programming language']. replace('Swift/Kotlin', 'Swift', inplace=True)
df['Your main technology / programming language']. replace('Python, Pytorch', 'Other', inplace=True)
df['Your main technology / programming language']. replace('scala', 'Other', inplace=True)
df['Your main technology / programming language']. replace('DWH', 'Other', inplace=True)
df['Your main technology / programming language']. replace('Java, Javascript', 'Java', inplace=True)
df['Your main technology / programming language']. replace('consumer analysis', 'Other', inplace=True)
df['Your main technology / programming language']. replace('Java Backend', 'Java', inplace=True)
df['Your main technology / programming language']. replace('Java, Kotlin', 'Java', inplace=True)
df['Your main technology / programming language']. replace('Java & Distributed Systems Stuff', 'Java', inplace=True)
df['Your main technology / programming language']. replace('nothing', 'Other', inplace=True)
df['Your main technology / programming language']. replace('JAVA', 'Java', inplace=True)
df['Your main technology / programming language']. replace('Pyrhon', 'Python', inplace=True)
df['Your main technology / programming language']. replace('Tricentis Tosca', 'Other', inplace=True)
df['Your main technology / programming language']. replace('C#/.NET', 'C#', inplace=True)
df['Your main technology / programming language']. replace('Autonomous Driving', 'Other', inplace=True)
df['Your main technology / programming language']. replace('C++/c', 'C++', inplace=True)
df['Your main technology / programming language']. replace('Charles', 'Other', inplace=True)
df['Your main technology / programming language']. replace('Qlik BI Tool, SQL', 'Other', inplace=True)
df['Your main technology / programming language']. replace('Java, angular, Aws', 'Java', inplace=True)
df['Your main technology / programming language']. replace('Azure', 'Other', inplace=True)
df['Your main technology / programming language']. replace('React/Typescript', 'Other', inplace=True)

In [104]:
df['Your main technology / programming language'].replace('Php', "PHP", inplace=True)
df['Your main technology / programming language'].replace('php', "PHP", inplace=True)
df['Your main technology / programming language'].replace('java', "Java", inplace=True)
df['Your main technology / programming language'].replace('python', "Python", inplace=True)
df['Your main technology / programming language'].replace('Javascript', "JavaScript", inplace=True)
df['Your main technology / programming language'].replace('javascript', "JavaScript", inplace=True)
df['Your main technology / programming language'].replace('Typescript', "TypeScript", inplace=True)
df['Your main technology / programming language'].replace('Android/Kotlin', "Kotlin", inplace=True)
df['Your main technology / programming language'].replace('JavaScript / typescript', "JavaScript", inplace=True)
df['Your main technology / programming language'].replace('Javascript / Typescript', "JavaScript", inplace=True)
df['Your main technology / programming language'].replace('Swift, Objective-C, iOS', "Swift", inplace=True)
df['Your main technology / programming language'].replace('Web developer', "Other", inplace=True)
df['Your main technology / programming language'].replace('DevOps', "Other", inplace=True)
df['Your main technology / programming language'].replace('none', "Other", inplace=True)
df['Your main technology / programming language'].replace('Pascal, VB.NET, C#', "Other", inplace=True)
df['Your main technology / programming language'].replace('Jira', "Other", inplace=True)
df['Your main technology / programming language'].replace('FBD', "Other", inplace=True)
df['Your main technology / programming language'].replace('NodeJS/TS', "Other", inplace=True)
df['Your main technology / programming language'].replace('Network Automation', "Other", inplace=True)
df['Your main technology / programming language'].replace('Oracle', "Other", inplace=True)
df['Your main technology / programming language'].replace('TypeScript, Kotlin', "TypeScript", inplace=True)
df['Your main technology / programming language'].replace('Clojure', "Other", inplace=True)
df['Your main technology / programming language'].replace('SAP BW / ABAP', "Other", inplace=True)
df['Your main technology / programming language'].replace('С#', "C#", inplace=True)
df['Your main technology / programming language'].replace('JavaScript/ES6', "JavaScript", inplace=True)
df['Your main technology / programming language'].replace('C++, Java, Embedded C', "C++", inplace=True)
df['Your main technology / programming language'].replace('Js', "JavaScript", inplace=True)
df['Your main technology / programming language'].replace('Agile', "Other", inplace=True)
df['Your main technology / programming language'].replace('Python, statistics, SQL', "Python", inplace=True)
df['Your main technology / programming language'].replace('NLP, Python', "Python", inplace=True)
df['Your main technology / programming language'].replace('Java/Scala', "Java", inplace=True)
df['Your main technology / programming language'].replace('Computer Networking,  Network Security', "Other", inplace=True)
df['Your main technology / programming language'].replace('Java / Scala', "Java", inplace=True)
df['Your main technology / programming language'].replace('Javascript, Angular', "JavaScript", inplace=True)
df['Your main technology / programming language'].replace('-', "Other", inplace=True)
df['Your main technology / programming language'].replace('Kotlin/PHP', "Kotlin", inplace=True)
df['Your main technology / programming language'].replace('GCP', "Other", inplace=True)
df['Your main technology / programming language'].replace('Js, TypeScript, Angular', "JavaScript", inplace=True)
df['Your main technology / programming language'].replace('Embedded C++', "C++", inplace=True)
df['Your main technology / programming language'].replace('.net', ".NET", inplace=True)
df['Your main technology / programming language'].replace('c++', "C++", inplace=True)
df['Your main technology / programming language'].replace('JS', "JavaScript", inplace=True)

df['Your main technology / programming language'].replace('go', "Go", inplace=True)
df['Your main technology / programming language'].replace('SRE', "Other", inplace=True)
df['Your main technology / programming language'].replace('Kubernetes, Terraform, GCP', "Kubernetes", inplace=True)
df['Your main technology / programming language'].replace('Spring', "Other", inplace=True)
df['Your main technology / programming language'].replace('--', "Other", inplace=True)

 Seniority level

In [105]:
df['Seniority level'].value_counts()

Senior                                       565
Middle                                       366
Lead                                         166
Junior                                        79
Head                                          44
Principal                                      3
Manager                                        1
C-level executive manager                      1
intern                                         1
Director                                       1
CTO                                            1
Self employed                                  1
Key                                            1
No level                                       1
C-Level                                        1
Intern                                         1
student                                        1
Work Center Manager                            1
no idea, there are no ranges in the firm       1
Working Student                                1
VP                  

In [106]:

df['Seniority level'].replace('Student', 'Entry level', inplace=True)
df['Seniority level'].replace('student', 'Entry level', inplace=True)
df['Seniority level'].replace('intern', 'Entry level', inplace=True)
df['Seniority level'].replace('Intern', 'Entry level', inplace=True)
df['Seniority level'].replace('Working Student', 'Entry level', inplace=True)
df['Seniority level'].replace('C-level executive manager', 'C-Level', inplace=True)
df['Seniority level'].replace('no idea, there are no ranges in the firm', 'Other', inplace=True)
df['Seniority level'].replace('No level', 'Other', inplace=True)
df['Seniority level'].replace('No level', 'Other', inplace=True)
df['Seniority level'].replace('Work Center Manager', 'Other', inplace=True)
df['Seniority level'].replace('Manager', 'Other', inplace=True)
df['Seniority level'].replace('CTO', 'Other', inplace=True)
df['Seniority level'].replace('Director', 'Other', inplace=True)
df['Seniority level'].replace('Key', 'Other', inplace=True)
df['Seniority level'].replace('Principal', 'Other', inplace=True)
df['Seniority level'].replace('Self employed', 'Other', inplace=True)
df['Seniority level'].replace('VP', 'Other', inplace=True)

Years of experience in Germany

In [107]:
df['Years of experience in Germany'].replace('< 1', 0.5, inplace=True)
df['Years of experience in Germany'].replace('4 (in Switzerland), 0 (in Germany)', 0, inplace=True)
df['Years of experience in Germany'].replace('4 month', 0.25, inplace=True)
df['Years of experience in Germany'].replace('2,5', 2.5, inplace=True)
df['Years of experience in Germany'].replace('3,5', 3.5, inplace=True)
df['Years of experience in Germany'].replace('1,5', 1.5, inplace=True)
df['Years of experience in Germany'].replace('4,5', 4.5, inplace=True)
df['Years of experience in Germany'].replace('0,5', 0.5, inplace=True)
df['Years of experience in Germany'].replace('1,7', 1.7, inplace=True)
df['Years of experience in Germany'].replace('3 months', 0.2, inplace=True)
df['Years of experience in Germany'].replace('3 (in Poland)', 0, inplace=True)
df['Years of experience in Germany'].replace('<1', 0.5, inplace=True)
df['Years of experience in Germany'].replace('⁰', 0, inplace=True)
df['Years of experience in Germany'].replace('0,3', 0.3, inplace=True)
df['Years of experience in Germany'].replace('-', 0, inplace=True)
df['Years of experience in Germany'].replace('6 (not as a data scientist, but as a lab scientist)', 0, inplace=True)
df['Years of experience in Germany'].replace('less than year', 0, inplace=True)

df['Years of experience in Germany'] = pd.to_numeric(df['Years of experience in Germany'], errors = 'coerce')

Position

In [108]:
df['Position '].value_counts()

Software Engineer                               387
Backend Developer                               174
Data Scientist                                  110
Frontend Developer                               89
QA Engineer                                      71
DevOps                                           57
Mobile Developer                                 53
ML Engineer                                      42
Product Manager                                  39
Data Engineer                                    25
Designer (UI/UX)                                 16
Engineering Manager                              10
Data Analyst                                      8
CTO                                               5
Project Manager                                   4
SRE                                               4
Software Architect                                4
Fullstack Developer                               4
Business Analyst                                  3
Team Lead   

city

In [109]:
df['City'].value_counts()

Berlin                   681
Munich                   236
Frankfurt                 44
Hamburg                   40
Stuttgart                 26
Cologne                   20
Amsterdam                  9
Stuttgart                  7
Karlsruhe                  6
Düsseldorf                 6
Prague                     6
Moscow                     6
Zurich                     5
Stockholm                  5
London                     5
Nürnberg                   4
Helsinki                   4
Heidelberg                 4
Düsseldorf                 4
Rome                       3
Mannheim                   3
Darmstadt                  3
Boeblingen                 3
Wolfsburg                  3
Bonn                       3
Hannover                   3
Dusseldorf                 3
Tallinn                    3
Milan                      2
Lisbon                     2
Vienna                     2
Regensburg                 2
Barcelona                  2
Eindhoven                  2
Saint-Petersbu

In [110]:
df['City'].replace('Amsterdam', 'Other', inplace=True)
df['City'].replace('Düsseldorf', 'Other', inplace=True)
df['City'].replace('Stuttgart', 'Other', inplace=True)
df['City'].replace('Stuttgart', 'Other', inplace=True)
df['City'].replace('Düsseldorf', 'Other', inplace=True)
df['City'].replace('Heidelberg', 'Other', inplace=True)
df['City'].replace('Mannheim', 'Other', inplace=True)
df['City'].replace('Dublin', 'Other', inplace=True)
df['City'].replace('Murnau am Staffelsee', 'Other', inplace=True)
df['City'].replace('Brussels', 'Other', inplace=True)
df['City'].replace('Hildesheim', 'Other', inplace=True)
df['City'].replace('Ingolstadt', 'Other', inplace=True)
df['City'].replace('Nuremberg', 'Other', inplace=True)
df['City'].replace('Braunschweig', 'Other', inplace=True)
df['City'].replace('Karlsruhe', 'Other', inplace=True)
df['City'].replace('Düsseldorf', 'Other', inplace=True)
df['City'].replace('Prague', 'Other', inplace=True)
df['City'].replace('Moscow', 'Other', inplace=True)
df['City'].replace('Karlsruhe', 'Other', inplace=True)
df['City'].replace('London', 'Other', inplace=True)
df['City'].replace('Stockholm', 'Other', inplace=True)
df['City'].replace('Zurich', 'Other', inplace=True)
df['City'].replace('Helsinki', 'Other', inplace=True)
df['City'].replace('Heidelberg', 'Other', inplace=True)
df['City'].replace('Düsseldorf', 'Other', inplace=True)
df['City'].replace('Nürnberg', 'Other', inplace=True)
df['City'].replace('Bonn', 'Other', inplace=True)
df['City'].replace('Darmstadt', 'Other', inplace=True)
df['City'].replace('Rome', 'Other', inplace=True)
df['City'].replace('Wolfsburg', 'Other', inplace=True)
df['City'].replace('Hannover', 'Other', inplace=True)
df['City'].replace('Mannheim', 'Other', inplace=True)
df['City'].replace('Dusseldorf', 'Other', inplace=True)
df['City'].replace('Boeblingen', 'Other', inplace=True)
df['City'].replace('Tallinn', 'Other', inplace=True)
df['City'].replace('Siegen', 'Other', inplace=True)
df['City'].replace('Milan', 'Other', inplace=True)
df['City'].replace('Saint-Petersburg', 'Other', inplace=True)
df['City'].replace('Regensburg', 'Other', inplace=True)
df['City'].replace('Madrid', 'Other', inplace=True)
df['City'].replace('Nuremberg', 'Other', inplace=True)
df['City'].replace('Rosenheim', 'Other', inplace=True)
df['City'].replace('Lisbon', 'Other', inplace=True)
df['City'].replace('Aachen', 'Other', inplace=True)
df['City'].replace('Leipzig', 'Other', inplace=True)
df['City'].replace('Vienna', 'Other', inplace=True)
df['City'].replace('Kyiv', 'Other', inplace=True)
df['City'].replace('Paris', 'Other', inplace=True)
df['City'].replace('Eindhoven', 'Other', inplace=True)
df['City'].replace('Warsaw', 'Other', inplace=True)
df['City'].replace('Ingolstadt', 'Other', inplace=True)
df['City'].replace('Barcelona', 'Other', inplace=True)
df['City'].replace('Minsk', 'Other', inplace=True)
df['City'].replace('Konstanz', 'Other', inplace=True)
df['City'].replace('Wroclaw', 'Other', inplace=True)
df['City'].replace('Köln', 'Other', inplace=True)
df['City'].replace('Cambridge', 'Other', inplace=True)
df['City'].replace('Tampere (Finland)', 'Other', inplace=True)
df['City'].replace('Dublin', 'Other', inplace=True)
df['City'].replace('Sofia', 'Other', inplace=True)
df['City'].replace('Kempten', 'Other', inplace=True)
df['City'].replace('Samara', 'Other', inplace=True)
df['City'].replace('Bielefeld', 'Other', inplace=True)
df['City'].replace('Istanbul', 'Other', inplace=True)
df['City'].replace('Jena', 'Other', inplace=True)
df['City'].replace('Tuttlingen', 'Other', inplace=True)
df['City'].replace('Luttich', 'Other', inplace=True)
df['City'].replace('Ulm', 'Other', inplace=True)
df['City'].replace('Kiev', 'Other', inplace=True)
df['City'].replace('Erlangen', 'Other', inplace=True)
df['City'].replace('Braunschweig', 'Other', inplace=True)
df['City'].replace('Heidelberg', 'Other', inplace=True)
df['City'].replace('Lübeck', 'Other', inplace=True)
df['City'].replace('Heilbronn', 'Other', inplace=True)
df['City'].replace('Moldova', 'Other', inplace=True)
df['City'].replace('Basel', 'Other', inplace=True)
df['City'].replace('Fr', 'Other', inplace=True)
df['City'].replace('Malta', 'Other', inplace=True)
df['City'].replace('Dublin', 'Other', inplace=True)
df['City'].replace('Copenhagen', 'Other', inplace=True)
df['City'].replace('Bodensee', 'Other', inplace=True)
df['City'].replace('Hildesheim', 'Other', inplace=True)
df['City'].replace('Marseille', 'Other', inplace=True)
df['City'].replace('Den Haag', 'Other', inplace=True)
df['City'].replace('Brunswick', 'Other', inplace=True)
df['City'].replace('Krakow', 'Other', inplace=True)
df['City'].replace('Sevilla', 'Other', inplace=True)
df['City'].replace('Duesseldorf', 'Other', inplace=True)
df['City'].replace('Cracovia', 'Other', inplace=True)
df['City'].replace('Salzburg', 'Other', inplace=True)
df['City'].replace('France', 'Other', inplace=True)
df['City'].replace('NJ, USA', 'Other', inplace=True)
df['City'].replace('Dresden', 'Other', inplace=True)
df['City'].replace('Bölingen', 'Other', inplace=True)
df['City'].replace('warsaw', 'Other', inplace=True)
df['City'].replace('Warsaw, Poland', 'Other', inplace=True)
df['City'].replace('City in Russia', 'Other', inplace=True)
df['City'].replace('Prefer not to say', 'Other', inplace=True)
df['City'].replace('Utrecht', 'Other', inplace=True)
df['City'].replace('Cupertino', 'Other', inplace=True)
df['City'].replace('Koblenz', 'Other', inplace=True)
df['City'].replace('Riga, Latvia', 'Other', inplace=True)
df['City'].replace('Würzburg', 'Other', inplace=True)
df['City'].replace('Friedrichshafen', 'Other', inplace=True)
df['City'].replace('Münster', 'Other', inplace=True)
df['City'].replace('Innsbruck', 'Other', inplace=True)
df['City'].replace('Ingolstadt', 'Other', inplace=True)
df['City'].replace('Hildesheim', 'Other', inplace=True)
df['City'].replace('Konstanz area', 'Other', inplace=True)
df['City'].replace('Brussels', 'Other', inplace=True)
df['City'].replace('Saarbrücken', 'Other', inplace=True)
df['City'].replace('Mannheim', 'Other', inplace=True)
df['City'].replace('Paderborn', 'Other', inplace=True)
df['City'].replace('Ansbach', 'Other', inplace=True)
df['City'].replace('Murnau am Staffelsee', 'Other', inplace=True)
df['City'].replace('Brussels', 'Other', inplace=True)
df['City'].replace('Karlsruhe', 'Other', inplace=True)
df['City'].replace('Dusseldurf', 'Other', inplace=True)
df['City'].replace('Milano', 'Other', inplace=True)
df['City'].replace('Bucharest', 'Other', inplace=True)
df['City'].replace('Schleswig-Holstein', 'Other', inplace=True)
df['City'].replace('Zürich', 'Other', inplace=True)
df['City'].replace('Cracow', 'Other', inplace=True)
df['City'].replace('Dortmund', 'Other', inplace=True)
df['City'].replace('Nuremberg', 'Other', inplace=True)
df['City'].replace('Walldorf', 'Other', inplace=True)

In [111]:
df['Yearly bonus + stocks in EUR'].fillna(0)

0       5.000000e+03
1       0.000000e+00
2       1.200000e+05
3       0.000000e+00
4       0.000000e+00
5       5.000000e+03
6       0.000000e+00
7       0.000000e+00
8       0.000000e+00
9       0.000000e+00
10      0.000000e+00
11      1.010000e+02
12      4.000000e+04
13      0.000000e+00
14      9.400000e+03
15      2.000000e+03
16      0.000000e+00
17      0.000000e+00
18      0.000000e+00
19      8.000000e+04
20      0.000000e+00
21      0.000000e+00
22      0.000000e+00
23      0.000000e+00
24      0.000000e+00
25      4.000000e+03
26      1.000000e+04
27      1.000000e+03
28      7.000000e+03
29      0.000000e+00
30      5.000000e+03
31      0.000000e+00
32      2.000000e+03
33      0.000000e+00
34      1.010000e+02
35      8.000000e+04
36      0.000000e+00
37      0.000000e+00
38      0.000000e+00
39      0.000000e+00
40      1.200000e+04
41      1.500000e+03
42      0.000000e+00
43      3.520000e+04
44      1.500000e+04
45      0.000000e+00
46      0.000000e+00
47      0.000

In [112]:
df.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Age,1226.0,NaN,NaN,NaN,32.509788,5.663804,20.0,29.0,32.0,35.0,69.0
Gender,1243,3,Male,1049,NaN,NaN,NaN,NaN,NaN,NaN,NaN
City,1253,18,Berlin,681,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Position,1247,148,Software Engineer,387,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Total years of experience,1230.0,NaN,NaN,NaN,9.07813,11.901317,0.0,5.0,8.0,12.0,383.0
Years of experience in Germany,1221.0,NaN,NaN,NaN,3.699386,3.641314,0.0,1.0,3.0,5.0,30.0
Seniority level,1241,10,Senior,565,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Your main technology / programming language,1126,41,Java,212,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Yearly brutto salary (without bonus and stocks) in EUR,1253.0,NaN,NaN,NaN,80279042.578715,2825061107.590495,10001.0,58800.0,70000.0,80000.0,99999999999.0
Yearly bonus + stocks in EUR,823.0,NaN,NaN,NaN,6096870.478068,174288300.185504,0.0,0.0,5000.0,20000.0,5000000000.0


In [113]:
df.isna().sum()

Age                                                        27
Gender                                                     10
City                                                        0
Position                                                    6
Total years of experience                                  23
Years of experience in Germany                             32
Seniority level                                            12
Your main technology / programming language               127
Yearly brutto salary (without bonus and stocks) in EUR      0
Yearly bonus + stocks in EUR                              430
Number of vacation days                                    77
Employment status                                          17
Сontract duration                                          29
Main language at work                                      16
Company size                                               18
Company type                                               25
dtype: i

In [114]:
df['Total Compensation'] = df['Yearly brutto salary (without bonus and stocks) in EUR'] + df['Yearly bonus + stocks in EUR']

In [115]:
df = df.drop(columns={'Yearly bonus + stocks in EUR', 'Yearly brutto salary (without bonus and stocks) in EUR'})

In [116]:
for col in ['Gender', 'City', 'Seniority level', 'Company type', 'Company size','Main language at work','Сontract duration','Your main technology / programming language', 'Employment status']:
    df[col] = df[col].astype('category')

In [117]:
#df = df.dropna()

In [130]:
df.replace('nan', np.nan, inplace=True)

In [118]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1253 entries, 0 to 1252
Data columns (total 15 columns):
 #   Column                                       Non-Null Count  Dtype   
---  ------                                       --------------  -----   
 0   Age                                          1226 non-null   float64 
 1   Gender                                       1243 non-null   category
 2   City                                         1253 non-null   category
 3   Position                                     1247 non-null   object  
 4   Total years of experience                    1230 non-null   float64 
 5   Years of experience in Germany               1221 non-null   float64 
 6   Seniority level                              1241 non-null   category
 7   Your main technology / programming language  1126 non-null   category
 8   Number of vacation days                      1176 non-null   float64 
 9   Employment status                            1236 non-null   ca

Model

In [136]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFECV
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import cross_val_score

model = RandomForestClassifier(n_jobs=-1)
pipe_model = LinearRegression()

min_features_to_select = 1 
rfecv_pipe = RFECV(
    estimator=pipe_model,
    step=1,
    cv=3,
    scoring="accuracy",
    min_features_to_select=min_features_to_select,
)

numeric_features = ['Age', 'Total years of experience', 'Number of vacation days','Years of experience in Germany']
numeric_transformer = Pipeline( steps=[
        ("imputer", SimpleImputer(strategy="mean")),
        ("scaler", MinMaxScaler())
    ])

categorical_features = ['Gender', 'City', 'Seniority level', 'Company type', 'Company size','Main language at work','Сontract duration','Your main technology / programming language', 'Employment status']
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer( remainder='drop',
        transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ])

#Split data as normal
y = df['Total Compensation']
X = df.drop(columns={'Total Compensation'})

#Build pipeline
l1_model = Lasso()
l2_model = Ridge()
pipeline_steps = [('pre', preprocessor),("fs", rfecv_pipe), ('model', l2_model) ]
pipe = Pipeline(pipeline_steps)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [137]:
pipe.fit(X_train, y_train)
pipe.score(X_test,y_test)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

# Answers and Explainations
<li> Combined Salary and Bonus into one and then dropped the 2 individual columns
<li>Dropped other technologies as it was redundant considering the answers were the same as main technologies.

### Results

### Feature Selection Activities

### Hyperparameter Changes